# [Ateliers: Technologies de l'intelligence Artificielle](https://github.com/wikistat/AI-Frameworks)

<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 
<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" width=400, style="max-width: 150px; display: inline"  alt="Wikistat"/></a>
<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" width=400,  style="float:right;  display: inline" alt="IMT"/> </a>
    
</center>

# Traitement Naturel du Langage (NLP) : Génération de Texte avec des Réseaux Récurrent. 

Au cours de ce calepin, nous allons voir comment générer des description de produits à l'aide de Réseaux Récurents et notamment grace aux structure LSTM (Long-Short Term Memory). 

L'intérêt de cette application est limité. Les descriptions de textes de ce document sont trop pauvres syntaxiquement pour pouvoir juger réellement de la qualité du texte généré. L'intérêt réel de ce calepin est de voir comment les données doivent être mis en forme pour être utilisé dans un réseau recurrent dans un but de génération de texte.

## Librairies

In [2]:
#Importation des librairies utilisées

import pandas as pd
import numpy as np
import pickle
import functools
from tqdm import tqdm

import keras.models as km
import keras.layers as kl

## Téléchargement des données

In [4]:
DATA_DIR = ""
X = np.load(DATA_DIR+"data/description_coque.npy")[:100000]
X.shape

(100000,)

## Mise en forme  des données

### Création de la liste des caractères

In [9]:
chars = list(functools.reduce(lambda x,y : x.union(y), [set(x) for x in X], set()))
print(chars, len(chars))

['O', 'f', '%', ' ', 'T', '&', 'V', '*', ')', '?', 'z', 'Q', '6', ':', 'X', 'C', 'g', 'N', 'I', 'm', 'u', 'U', '-', '3', 'j', 'w', 'ô', '!', 'c', 'ê', 'i', 'b', 'd', 's', 'l', 'h', 'a', 'A', 'L', 'à', 'Y', '8', 'R', 'F', 'é', 'â', 'S', '"', 'Z', 'D', 'M', 'x', 'J', '\xa0', 'P', 'k', 'è', 'K', '4', '.', "'", '7', ',', 'r', 'v', '5', '2', '1', 't', 'G', '+', 'B', 'y', 'e', 'q', '9', '(', 'E', 'W', '/', '0', 'o', 'H', 'p', 'ç', '…', 'n'] 87


In [10]:
chars.extend(["start","end"])

### Création des dictionnaires

In [11]:
int_to_char = {i:c for i,c in enumerate(chars)}
char_to_int = {c:i for i,c in int_to_char.items()}
I_START = char_to_int["start"]
I_END = char_to_int["end"]

### Paramètre du modèle 

In [12]:
SIZE_VOCAB = len(chars)
LENGTH_SEQUENCE = len(X[0])+1
N_X = 100000
print("Number of sentences : %d, Vocab size : %d, Sequence Length : %d" %(N_X, SIZE_VOCAB, LENGTH_SEQUENCE))

Number of sentences : 100000, Vocab size : 89, Sequence Length : 198


### Encodage des Descriptions


#### Définition des fonctions

In [13]:
def encode_input_output_sequence(x, length_sequence, size_vocab, char_to_int_dic, i_start, i_end):
    n = x.shape[0]
    x_vec = np.zeros((n,length_sequence, size_vocab))
    y_vec = np.zeros((n,length_sequence, size_vocab))
    x_vec[:,0,i_start] = 1
    y_vec[:,-1,i_end] = 1
    for ix,x in tqdm(enumerate(x)):
        for ic,c in enumerate(x):
            c_int = char_to_int_dic[c]
            x_vec[ix,ic+1,c_int]=1
    y_vec[:,:-1,:] = x_vec[:,1:,:] 
    return x_vec, y_vec


def decode_sequence(x, int_to_char_dic):
    seq = []
    for i in np.where(x)[1]:
        seq.append(int_to_char_dic[i])
    return "".join(seq)


#### Quelques exemples

In [14]:
x_vec, y_vec = encode_input_output_sequence(X[:N_X], LENGTH_SEQUENCE, SIZE_VOCAB, char_to_int,I_START,I_END)

100000it [00:07, 12644.67it/s]


In [15]:
print(decode_sequence(x_vec[8], int_to_char))
print(decode_sequence(y_vec[8], int_to_char))


startPour samsung galaxy s2 - i9100 : coque decor drapeau us vintage - Cette coque fantaisie protège et habille votre SAMSUNG Galaxy S2 - i9100. Parfaitement adaptée, elle permet l… Voir la présentation
Pour samsung galaxy s2 - i9100 : coque decor drapeau us vintage - Cette coque fantaisie protège et habille votre SAMSUNG Galaxy S2 - i9100. Parfaitement adaptée, elle permet l… Voir la présentationend


**Exercice** Retrouvez la phrase originale à partir de la phrase encodé. Vérifiez que x et y sont bien les mêmes fonciton d'écalé d'un index

## Apprentissage

In [18]:
nb_hidden = 32
epochs = 10
batch_size=128

model = km.Sequential()
model.add(kl.LSTM(nb_hidden, input_shape=(None, SIZE_VOCAB), return_sequences=True))
model.add(kl.TimeDistributed(kl.Dense(SIZE_VOCAB)))
model.add(kl.Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, None, 32)          15616     
_________________________________________________________________
time_distributed_2 (TimeDist (None, None, 89)          2937      
_________________________________________________________________
activation_2 (Activation)    (None, None, 89)          0         
Total params: 18,553
Trainable params: 18,553
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")
model.fit(x_vec, y_vec, epochs=epochs, batch_size=batch_size)

Epoch 1/10
100000/100000 [==============================] - 170s 2ms/step - loss: 2.7328
Epoch 2/10
100000/100000 [==============================] - 169s 2ms/step - loss: 1.4591
Epoch 3/10
100000/100000 [==============================] - 170s 2ms/step - loss: 0.8730
Epoch 4/10
100000/100000 [==============================] - 170s 2ms/step - loss: 0.6003
Epoch 5/10
100000/100000 [==============================] - 170s 2ms/step - loss: 0.4592
Epoch 6/10
100000/100000 [==============================] - 170s 2ms/step - loss: 0.3845
Epoch 7/10
100000/100000 [==============================] - 170s 2ms/step - loss: 0.3421
Epoch 8/10
100000/100000 [==============================] - 170s 2ms/step - loss: 0.3168
Epoch 9/10
100000/100000 [==============================] - 170s 2ms/step - loss: 0.3013
Epoch 10/10
100000/100000 [==============================] - 171s 2ms/step - loss: 0.2899


In [30]:
model.save("data/PLOPP.h5")

In [20]:
i_test = 0
print(decode_sequence(x_vec[i_test], int_to_char))

startPour apple iphone 4 : coque bumper silicone blanc - Cet étui en silicone rigide protège et habille votre APPLE iPhone 4. Parfaitement adapté, il permet l'accès à toutes les fo… Voir la présentation


In [33]:
x_pred = np.zeros((1, LENGTH_SEQUENCE, SIZE_VOCAB))
print("step 0")
x_pred[0,0,I_START] =1
x_pred_str = decode_sequence(x_pred[0], int_to_char)
print(x_pred_str)
for i in range(LENGTH_SEQUENCE-1):
    ix = np.argmax(model.predict(x_pred[:,:i+1,:])[0][-1,:])
    x_pred[0,i+1,ix] = 1
    x_pred_str=decode_sequence(x_pred[0], int_to_char)
    print(x_pred_str, end="\r")

step 0
start
